### Assignment - 5
#### execute the code in python-3 along with weights_final.h5py file

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

/Users/avni/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def one_hot_encoding(total_sample, input_data, numbers):
    y = np.zeros((total_sample, numbers))
    for i in range(total_sample):
        position = input_data[i][0]
        y[i][position] = 1
    return y

In [3]:
data_y = np.load('ex5_train_y.npy')
data_y

array([4, 4, 4, ..., 4, 3, 2])

In [4]:
Y = np.reshape(data_y, (data_y.shape[0],1))
y = one_hot_encoding(1020, Y, 6)
y.shape

(1020, 6)

In [5]:
data_x = np.load('ex5_train_x.npy')
X_norm = data_x/255 - 0.5 #normalizing data_x

In [6]:
def compute_cost(m, A2, y):
    new_cost = (-1/m) * np.sum(np.multiply(y, np.log(A2)) + np.multiply(1 - y, np.log(1 - A2)))
    return new_cost
    

In [7]:
def sigmoid(Z):     
    return 1/ (1 + np.exp(-Z))

In [8]:
def relu(Z):
    return (Z * (Z > 0))

In [9]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
      
    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), 'constant', constant_values=0)
   
    
    return X_pad

In [10]:
def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, result of convolving the sliding window (W, b) on a slice x of the input data
    """

    s = np.multiply(a_slice_prev, W) + b
  
    Z = np.sum(s)

    return Z

In [11]:
def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    
    # dimensions from A_prev's shape 
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # dimensions from W's shape 
    (f, f, n_C_prev, n_C) = W.shape

    # information from "hparameters" 
    stride = hparameters['stride']
    pad = hparameters['pad']

    
    # dimensions of the CONV output 
    n_H = int((n_H_prev - f + 2 * pad) / stride) + 1
    n_W = int((n_W_prev - f + 2 * pad) / stride) + 1
    
    # Initialize the output volume Z with zeros. 
    Z = np.zeros((m, n_H, n_W, n_C))
    A = np.zeros((m, n_H, n_W, n_C))
    
    # Creating A_prev_pad by padding A_prev
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):   
#         print "i= ", i                               # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i,:]                   # Select ith training example's padded activation
        for h in range(n_H):                           # loop over vertical axis of the output volume
            for w in range(n_W):                       # loop over horizontal axis of the output volume
                for c in range(n_C):                   # loop over channels (= #filters) of the output volume
                   
                    vert_start = h * stride
                    vert_end = h*stride + f
                    horiz_start = w * stride
                    horiz_end =w*stride + f
                    # Use the corners to define the (3D) slice of a_prev_pad 
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. 
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[...,c], b[...,c])
#                     print Z.shape
                    # activation of convolution layer using RELU
                    A[i, h, w, c] = relu(Z[i, h, w, c])
                    
    
    # Saving information in "cache" for the back propagation
    cache = (A_prev, W, b, hparameters)
    
    return Z, A, cache

In [12]:
def pool_forward(A_prev, hparameters, mode ):
    """
    Implements the forward pass of the pooling layer
    
    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
    """
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
  
    f = hparameters["f"]
    stride = hparameters["stride"]
    

    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    

    A = np.zeros((m, n_H, n_W, n_C))  
    P = np.zeros((m, n_H, n_W, n_C))
    
   
    for i in range(m):                           # loop over the training examples
        for h in range(n_H):                     # loop on the vertical axis of the output volume
            for w in range(n_W):                 # loop on the horizontal axis of the output volume
                for c in range (n_C):            # loop over the channels of the output volume
                    
                 
                    vert_start = h * stride
                    vert_end = h * stride + f
                    horiz_start = w * stride
                    horiz_end = w * stride + f
                    
                   
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
                    
                   
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "avg":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
    
    
    # storing the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)

    
    return A , cache

In [13]:
def fully_connected(W1,W2,X,b1,b2):
    
    Z1 = np.add(np.dot(W1, X.T) , b1) 
    A1 = relu(Z1)
    Z2 = np.add(np.dot(W2,A1), b2)
    A2 = sigmoid(Z2)   
    return Z1, A1, Z2, A2

In [14]:
# loading the latest weights from weight_final.hdf5 
weights_file = h5py.File("weights_final.hdf5", "r")
W_conv1 = np.array(weights_file["W1"])
b_conv1 = np.array(weights_file["b1"])
W_conv2 = np.array(weights_file["W2"])
b_conv2 = np.array(weights_file["b2"])
W1 = np.array(weights_file["W3"])
b1 = np.array(weights_file["b3"])
W2 = np.array(weights_file["W4"])
b2 = np.array(weights_file["b4"])
print (W_conv1.shape)
print (b_conv1.shape)
print (W_conv2.shape)
print (b_conv2.shape)
print (W1.shape)
print (b1.shape)
print (W2.shape)
print (b2.shape)

(4, 4, 3, 8)
(1, 1, 1, 8)
(4, 4, 8, 16)
(1, 1, 1, 16)
(108, 1296)
(108, 1)
(6, 108)
(6, 1)


In [15]:
def sigmoid_prime(x):
    return x * (1 - x)

In [16]:
def relu_prime(x):
    return 1. * (x > 0)

In [17]:
def conv_backward(dZ, cache):
    """
    Implement the backward propagation for a convolution function
    
    Arguments:
    dZ -- gradient of the cost with respect to the output of the conv layer (Z), numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward(), output of conv_forward()
    
    Returns:
    dA_prev -- gradient of the cost with respect to the input of the conv layer (A_prev),
               numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    dW -- gradient of the cost with respect to the weights of the conv layer (W)
          numpy array of shape (f, f, n_C_prev, n_C)
    db -- gradient of the cost with respect to the biases of the conv layer (b)
          numpy array of shape (1, 1, 1, n_C)
    """
    
  
    (A_prev, W, b, hparameters) = cache
    
 
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
 
    (f, f, n_C_prev, n_C) = W.shape
    
   
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    

    (m, n_H, n_W, n_C) = dZ.shape
    

    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):                       # loop over the training examples
        
        # select ith training example from A_prev_pad and dA_prev_pad
        a_prev_pad = A_prev_pad[i]
        da_prev_pad = dA_prev_pad[i]
        
        for h in range(n_H):                   # loop over vertical axis of the output volume
            for w in range(n_W):               # loop over horizontal axis of the output volume
                for c in range(n_C):           # loop over the channels of the output volume
                    
                
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                 
                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

               
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                    
        if(pad == 0):
            dA_prev [i,:,:,:] = da_prev_pad [:,:,:]
            
        elif(pad > 0):
            
            dA_prev [i,:,:,:] = da_prev_pad [pad :- pad ,pad : -pad,:]
    
    
    return dA_prev, dW, db

In [18]:
def create_mask_from_window(x):
    """
    Creates a mask from an input matrix x, to identify the max entry of x.
    
    Arguments:
    x -- Array of shape (f, f)
    
    Returns:
    mask -- Array of the same shape as window, contains a True at the position corresponding to the max entry of x.
    """ 
    mask = x == np.max(x)    
    return mask

In [19]:
def distribute_value(dz, shape):
    """
    Distributes the input value in the matrix of dimension shape
    
    Arguments:
    dz -- input scalar
    shape -- the shape (n_H, n_W) of the output matrix for which we want to distribute the value of dz
    
    Returns:
    a -- Array of size (n_H, n_W) for which we distributed the value of dz
    """


    (n_H, n_W) = shape
    
  
    average = dz / (n_H * n_W)
 
    a = np.ones(shape) * average
  
    
    return a

In [20]:
def pool_backward(dA, cache, mode):
    """
    Implements the backward pass of the pooling layer
    
    Arguments:
    dA -- gradient of cost with respect to the output of the pooling layer, same shape as A
    cache -- cache output from the forward pass of the pooling layer, contains the layer's input and hparameters 
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    dA_prev -- gradient of cost with respect to the input of the pooling layer, same shape as A_prev
    """
    

    (A_prev, hparameters) = cache
    

    stride = hparameters["stride"]
    f = hparameters["f"]
    
   
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape
 
    dA_prev = np.zeros(A_prev.shape)
    
    for i in range(m):                       # loop over the training examples
        # select training example from A_prev
        a_prev = A_prev[i]
        for h in range(n_H):                   # loop on the vertical axis
            for w in range(n_W):               # loop on the horizontal axis
                for c in range(n_C):           # loop over the channels (depth)
                  
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                
                    if mode == "max":
               
                        a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                      
                        mask = create_mask_from_window(a_prev_slice)
                       
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += np.multiply(mask, dA[i, h, w, c])
                        
                    elif mode == "avg":
                      
                        da = dA[i, h, w, c]
                 
                        shape = (f, f)
                      
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += distribute_value(da, shape)
                        

    
    return dA_prev

In [21]:
def gradient_descent(W1, b1, db1, dW1, W2, b2, db2, dW2, alpha):   
    W1_new = W1 - (alpha * dW1)
    b1_new = b1 - (alpha * db1)
    W2_new = W2 - (alpha * dW2)
    b2_new = b2 - (alpha * db2)
    return W1_new, b1_new, W2_new, b2_new

In [22]:
def backward_propagation_neural_network(m, A2,A1,W1, W2, X, y):
    dZ2 = A2 - y.T  # shape (6,1020)
    
    dW2 = (1/m)*(np.dot(dZ2 , A1.T))   # shape (6, 108)
    db2 = (1/m)*(np.sum(dZ2, axis = 1, keepdims = True)) # shape (6,1)
        
    dA1= np.dot(dZ2.T, W2)   # shape (108,1020)
    dZ1 = np.multiply(dA1.T, np.multiply(A1, (1- A1))) # shape (108, 1020)
    dW1 = (1/m)*(np.dot(dZ1, X))    # shape (108, 1296)
    db1 = (1/m)*(np.sum(dZ1, axis = 1, keepdims = True))  # shape (108,1)
    
    
    dA2 = np.dot(dZ1.T, W1) #shape (1020, 1296)
    return dZ1, dZ2, dW1, dW2, db1, db2, dA2
        

In [23]:
epochs=1
iters = 0
alpha = 1
for iters in range(epochs):
    hparameters_conv1 = {"pad": 1, "stride":2 }
    Z1, A1 , cache_conv1 = conv_forward(X_norm, W_conv1, b_conv1, hparameters_conv1)
    print ("Z1", Z1.shape)

    hparameters_pool1 = {"f": 5 , "stride": 1 }

    P1, cache_pool1 = pool_forward(A1, hparameters_pool1, mode = "max")

    print ("P1", P1.shape)

    hparameters_conv2 = {"pad": 0, "stride":2}

    Z2, A2 , cache_conv2 = conv_forward(P1, W_conv2, b_conv2, hparameters_conv2)

    print ("Z2", Z2.shape)

    hparameters_pool2 = {"f": 5 , "stride": 1 }

    P2, cache_pool2 = pool_forward(A2, hparameters_pool2, mode = "avg")

    print ("P2.shape", P2.shape)
    F3 = []
    for i in range (P2.shape[0]):
        mat = P2[i]
        mat_i = mat.flatten()
        F3.append(mat_i)
    F3= np.asarray(F3) 
    print ("F3.shape", F3.shape)
    m = y.shape[0]
    Z1, A1, Z2, A2 = fully_connected(W1,W2,F3,b1,b2)   
    "Cost calculation"
    A2T= A2.T
    cost = (-1/m) * np.sum(np.multiply(y, np.log(A2T)) + np.multiply(1 - y, np.log(1 - A2T)))
    print ("Cost =", cost)
    dZ1, dZ2, dW1, dW2, db1, db2, dA2 = backward_propagation_neural_network(m, A2,A1,W1, W2, F3, y)
    print ("dZ1.shape =", dZ1.shape)
    print ("dZ2.shape =", dZ2.shape)
    print ("dW1.shape =", dW1.shape)
    print ("dW2.shape =", dW2.shape)
    print ("db1.shape =", db1.shape)
    print ("db2.shape =", db2.shape)
    
    W1_new, b1_new, W2_new, b2_new= gradient_descent(W1, b1, db1, dW1, W2, b2, db2, dW2, alpha)
    dA2= np.reshape(dA2, (dA2.shape[0],9,9,16))
    print ("dA2.shape =", dA2.shape)
    dA_conv2 = pool_backward(dA2, cache_pool2, mode= "avg")
    print ("dA_conv2.shape =", dA_conv2.shape)
    dZ_conv2 = relu_prime(dA_conv2)
    print ("dZ_conv2.shape", dZ_conv2.shape)
    dP1, dW_conv2, db_conv2 = conv_backward(dZ_conv2, cache_conv2)
    print ("dP1 shape =", dP1.shape)
    print ("dW_conv1 shape =", dW_conv2.shape)
    print ("db_conv1 shape =", db_conv2.shape)
    dA_conv1 = pool_backward(dP1, cache_pool1, mode= "max")
    print("dA_conv1 ",dA_conv1.shape)
    dZ_conv1 = relu_prime(dA_conv1)
    print("dZ_conv1 shape ",dZ_conv1.shape)
    dX, dW_conv1, db_conv1 = conv_backward(dZ_conv1, cache_conv1)
    print("dX shape =",dX.shape)
    print("dW_conv1 shape =",dW_conv1.shape)
    print("db_conv1 shape =",db_conv1.shape)
    
    W_conv1,W_conv2,b_conv1,b_conv2 = gradient_descent(W_conv1, b_conv1, db_conv1, dW_conv1, W_conv2, b_conv2, db_conv2, dW_conv2, alpha)

Z1 (1020, 32, 32, 8)
P1 (1020, 28, 28, 8)
Z2 (1020, 13, 13, 16)
P2.shape (1020, 9, 9, 16)
F3.shape (1020, 1296)
Cost = 2.703270364601317
dZ1.shape = (108, 1020)
dZ2.shape = (6, 1020)
dW1.shape = (108, 1296)
dW2.shape = (6, 108)
db1.shape = (108, 1)
db2.shape = (6, 1)
dA2.shape = (1020, 9, 9, 16)
dA_conv2.shape = (1020, 13, 13, 16)
dZ_conv2.shape (1020, 13, 13, 16)
dP1 shape = (1020, 28, 28, 8)
dW_conv1 shape = (4, 4, 8, 16)
db_conv1 shape = (1, 1, 1, 16)
dA_conv1  (1020, 32, 32, 8)
dZ_conv1 shape  (1020, 32, 32, 8)
dX shape = (1020, 64, 64, 3)
dW_conv1 shape = (4, 4, 3, 8)
db_conv1 shape = (1, 1, 1, 8)


In [24]:
print ("Cost =", cost)

Cost = 2.703270364601317


In [25]:
Score = (10-cost)*10
print ("Score =", Score)

Score = 72.96729635398682


#### Cost = 2.703270364601317